# Dual Tomographic Compression (DTC): Analysis

## Gibbs randomness-compression proposition: An eﬃcient deep learning
    Mehmet Suezen  
    (c) 2025   
    License  Apache 2.0

In [ ]:
import dill
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
d2c_data =  dill.load(open('d2c_data.dill', "rb"))

In [ ]:
magnitute_prune_data = d2c_data['magnitute_prune_data']
random_prune_data = d2c_data['random_prune_data']
cs_prune_data = d2c_data['cs_prune_data']

In [ ]:
nrepeat = 1 
levels = np.array(magnitute_prune_data[0][0])
magnitute_prune_performance = np.array(
    [magnitute_prune_data[i][1] for i in range(nrepeat)]
)
random_prune_performance = np.array([random_prune_data[i][1] for i in range(nrepeat)])
cs_prune_performance = np.array([cs_prune_data[i][1] for i in range(nrepeat)])

magnitute_prune = np.mean(magnitute_prune_performance, axis=0)
magnitute_prune_se = np.std(magnitute_prune_performance, axis=0) / np.sqrt(nrepeat)
random_prune = np.mean(random_prune_performance, axis=0)
random_prune_se = np.std(random_prune_performance, axis=0) / np.sqrt(nrepeat)
cs_prune = np.mean(cs_prune_performance, axis=0)
cs_prune_se = np.std(cs_prune_performance, axis=0) / np.sqrt(nrepeat)

In [ ]:
magnitute_prune_s = sm.nonparametric.lowess(exog=levels, endog=magnitute_prune, frac=0.3)
random_prune_s = sm.nonparametric.lowess(exog=levels, endog=random_prune, frac=0.3)
cs_prune_s = sm.nonparametric.lowess(exog=levels, endog=cs_prune, frac=0.3)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.constrained_layout.use'] = True
matplotlib.use('agg')
font = {"family": "normal", "weight": "bold", "size": 16}
plt.rc("font", **font)


In [ ]:

plt.close()
plt.plot(levels, magnitute_prune, "o", label="Magnitute Only Raw")
plt.plot(magnitute_prune_s[:, 0], magnitute_prune_s[:, 1], "--",  label="Magnitute Only")
plt.plot(levels, random_prune, "x", label="Random prune raw")
plt.plot(random_prune_s[:, 0], random_prune_s[:, 1], "-",  label="Random prune")
plt.plot(levels, cs_prune, "*", label="Magnitute with CS raw")
plt.plot(cs_prune_s[:, 0], cs_prune_s[:, 1], "-.",  label="Magnitute with CS")

plt.legend()

plt.xlabel(f"Percent Neurons removed: ")
plt.ylabel("Test Accuracy")
plt.title(f" Iterated Dual Tomographic Compression \n Advantage of Reconstruction ")
plt.savefig(f"dtc_accuracy.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
plt.close()
plt.plot(levels, magnitute_prune, "o", label="Magnitute Only Raw")
plt.plot(magnitute_prune_s[:, 0], magnitute_prune_s[:, 1], "--",  label="Magnitute Only")
plt.plot(levels, random_prune, "x", label="Random prune raw")
plt.plot(random_prune_s[:, 0], random_prune_s[:, 1], "-",  label="Random prune")
plt.plot(levels, cs_prune, "*", label="Magnitute with CS raw")
plt.plot(cs_prune_s[:, 0], cs_prune_s[:, 1], "-.",  label="Magnitute with CS")

plt.legend()
plt.xlim([0.8, 0.999])

plt.xlabel(f"Percent Neurons removed: ")
plt.ylabel("Test Accuracy")
plt.title(f" Iterated Dual Tomographic Compression \n Advantage of Reconstruction ")
plt.savefig(f"dtc_accuracy_closeup.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
import pandas as pd

df_performance = pd.DataFrame(
    {
        "sparsity": magnitute_prune_s[:, 0],
        "magnitute_prune": magnitute_prune_s[:, 1],
        "random_prune": random_prune_s[:, 1],
        "cs_prune": cs_prune_s[:, 1],
    }
)
df_performance


## Weight rays  

In [ ]:
cs_prune_data = d2c_data['cs_prune_data']

In [ ]:
#
# Evolution of total weight reconstructions.
# 
# total reconstructed weights: 
# weight_rays 
#
# Total weights : I/O hidden layers.
#
levels = cs_prune_data[0][0] # sparsity levels
ll = len(levels)
weight_rays_absmean = [] 
for j in range(1): # repeats
     weight_rays = [cs_prune_data[j][2][i] for i in range(ll)]
     weight_rays_absmean.append([np.mean(np.abs(ray))/len(ray) for ray in weight_rays])
weight_rays_absmean = np.mean(weight_rays_absmean, axis=0)

In [ ]:
plt.close()
plt.step(levels, weight_rays_absmean)
plt.xlabel(f"Percent Neurons removed: ")
plt.ylabel("Absolute Mean of Weight Ray \n Scaled with size")
plt.title(f" Weight Rays Absolute Mean ")
plt.savefig(f"dtc_weight_rays.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
#
# Gibbs randomness-compression proposition 
# 
levels = cs_prune_data[0][0] # sparsity levels
ll = len(levels)
theta1s = cs_prune_data[0][5] # CS measurement vectors
theta2s = cs_prune_data[0][6]


In [ ]:

for i in [5, 10, 15, 20]:
    y, x = np.histogram(np.ndarray.flatten((theta1s[i])), bins=np.arange(-50.0,50.0,0.2))
    u = y/np.sum(y)
    plt.step(x[1:], u, label=f"Sparsity level {levels[i]:.2f}")
plt.xlabel(f"CS Measurement vector states ")
plt.ylabel("Probability of occurance")
plt.title(f" CS measurement occurance probabilities ")
plt.legend()
plt.savefig(f"dtc_cs_measure_prob_y1.eps", format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
gibbs_entropy = []
for i in range(ll):
    y, x = np.histogram(np.ndarray.flatten((theta1s[i])), bins=np.arange(-50.0,50.0,0.2))
    u = y/np.sum(y)
    gibbs_entropy.append(-1.0*np.sum([p * np.log2(p+1e-10) for p in u]))
plt.plot(levels, gibbs_entropy, 'o')
plt.xlabel(f"Percent Neurons removed ")
plt.ylabel("Gibbs Entropy (bits)")
plt.title(f" Evolution of Entropy \n of \n Measurement vectors previous layer ")
plt.savefig(f"dtc_cs_gibbs_y1.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
# Pearson correlation between random performance and Gibbs entropy
np.corrcoef(gibbs_entropy, df_performance['cs_prune'])[0,1], np.corrcoef(gibbs_entropy, df_performance['random_prune'])[0,1]

In [ ]:

for i in [5, 10, 15, 20]:
    y, x = np.histogram(np.ndarray.flatten((theta2s[i])), bins=np.arange(-50.0,50.0,0.2))
    u = y/np.sum(y)
    plt.step(x[1:], u, label=f"Sparsity level {levels[i]:.2f}")
plt.xlabel(f"CS Measurement vector states ")
plt.ylabel("Probability of occurance")
plt.title(f" CS measurement occurance probabilities ")
plt.legend()
plt.savefig(f"dtc_cs_measure_prob_y2.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
gibbs_entropy = []
for i in range(ll):
    y, x = np.histogram(np.ndarray.flatten((theta2s[i])), bins=np.arange(-50.0,50.0,0.2))
    u = y/np.sum(y)
    gibbs_entropy.append(-1.0*np.sum([p * np.log(p+1e-10) for p in u]))
plt.plot(levels, gibbs_entropy, 'o')
plt.xlabel(f"Percent Neurons removed ")
plt.ylabel("Gibbs Entropy (bits)")
plt.title(f" Evolution of Entropy \n of \n Measurement vectors next layer ")
plt.savefig(f"dtc_cs_gibbs_y2.eps",  format="eps", dpi=1000, bbox_inches="tight")  
plt.close()

In [ ]:
np.corrcoef(gibbs_entropy, df_performance['cs_prune'])[0,1], np.corrcoef(gibbs_entropy, df_performance['random_prune'])[0,1]